# Predict for one sentence

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

language = "fr"

model = AutoModelForTokenClassification.from_pretrained("models/hidden_vs_public_fr")
tokenizer = AutoTokenizer.from_pretrained("models/hidden_vs_public_fr")
tokenizer.add_tokens(["[PERS]"])

In [ ]:
def get_labels(output):
    list_entities = []
    dict_labels = {"LABEL_0": "O",
                   "LABEL_1": "B-PUBLIC",
                   "LABEL_2": "I-PUBLIC",
                   "LABEL_3": "B-HIDDEN",
                   "LABEL_4": "I-HIDDEN",
                  }
    for entity_group in output:
        if entity_group["word"] == "[PERS]":
            list_entities.append(dict_labels[entity_group["entity_group"]])
    return list_entities

In [ ]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
output = ner_pipeline("Le président [PERS] a été invité à la conference de Paris.")
get_labels(output)

In [ ]:
output = ner_pipeline("[PERS] [PERS], menuisier, 37 ans, a été cambriolé.")
get_labels(output)

In [ ]:
output = ner_pipeline("Le président [PERS] a assisté à la conférence à Paris.")
get_labels(output)

In [ ]:
output = ner_pipeline("[PERS] [PERS], 37 ans, a été poignardé alors qu'il se rendait à l'opéra.")
# output = ner_pipeline("Le [PERS] [PERS] a utilisé l'occassion pour insister sur la securité publique à la chambre des députés.")
get_labels(output)

In [ ]:
output = ner_pipeline("Mr. [PERS] est le cousin du menuisier.")
get_labels(output)

In [ ]:
output = ner_pipeline("[PERS], de la Côte-aux-Fées, fusilier dans la compagnie Germann, et [PERS], fusilier, natif de Neuchâtel ; sont prévenus que s’ils estiment pouvoir prétendre une part aux susdits dons, ils doivent adresser le plutôt possible à la Chancellerie soussignée, les certificats qui constatent les blessures qu’ils ont reçues et les droits qu’ils ont de participer à ces dons, afin que ces pièces puissent être envoyées avant le 15 juillet au comité prémentionné siégant à Berne.")
get_labels(output)

In [ ]:
output = ner_pipeline("Le roi, la reine, le prince héritier, la princesse Louise, le prince Waldemar et sa femme se sont rendus d’hui à bord du vapeur Tantallon-Castle, qui est ancré dans le port de Copenhague, pour rendre visite à [PERS].")
get_labels(output)

In [ ]:
output = ner_pipeline("[PERS], ministre d’Etat belge et chef de la vieille-droite, dont nous avons annoncé le décès à l’âge de 86 ans, était une personnalité de premier plan.")
get_labels(output)

In [ ]:
output = ner_pipeline("[PERS], chef de la vieille-droite, dont nous avons annoncé le décès à l’âge de 86 ans, était une personnalité de premier plan.")
get_labels(output)

In [ ]:
output = ner_pipeline("Dans le conflit qui a éclaté entre [PERS] et [PERS], celui-ci a vraiment abusé de sa position.")
get_labels(output)

In [ ]:
output = ner_pipeline("Monsieur et Madame [PERS] [PERS] et leur fils [PERS]")
get_labels(output)

In [ ]:
output = ner_pipeline("Le télégraphe nons apportait lundi malin czar, Alexandre III, a été clamé immédiatement ; c’est le second fils d [PERS] [PERS]")
get_labels(output)

In [ ]:
output = ner_pipeline("M. [PERS] [PERS], mineur, habitant Garnich, a été happé par un convoi de wagonnets Rodange. Le président [PERS] [PERS] a assisté aux funérailles du mineur.")
get_labels(output)     

In [ ]:
output = ner_pipeline("Le Président [PERS] [PERS] a assisté aux funérailles  de M. [PERS] [PERS], mineur, habitant Garnich, a été happé par un convoi de wagonnets Rodange.")
get_labels(output)

# On sentences from the BNL

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

language = "fr"

model_person = AutoModelForTokenClassification.from_pretrained("models/person_fr")
tokenizer_person = AutoTokenizer.from_pretrained("models/person_fr")
pipeline_person = pipeline("ner",
                           model=model_person,
                           tokenizer=tokenizer_person,
                           aggregation_strategy="simple")

model_hidden_vs_public = AutoModelForTokenClassification.from_pretrained("models/hidden_vs_public_fr")
tokenizer_hidden_vs_public = AutoTokenizer.from_pretrained("models/hidden_vs_public_fr")
tokenizer_hidden_vs_public.add_tokens(["[PERS]"])
pipeline_hidden_vs_public = pipeline("ner",
                           model=model_hidden_vs_public,
                           tokenizer=tokenizer_hidden_vs_public,
                           aggregation_strategy="simple")

In [ ]:
def get_labels(output):
    list_entities = []
    dict_labels = {"LABEL_0": "O",
                   "LABEL_1": "B-PUBLIC",
                   "LABEL_2": "I-PUBLIC",
                   "LABEL_3": "B-HIDDEN",
                   "LABEL_4": "I-HIDDEN",
                  }
    for entity_group in output:
        if entity_group["word"] == "[PERS]":
            list_entities.append(dict_labels[entity_group["entity_group"]])
    return list_entities

In [ ]:
from ast import literal_eval
from tqdm import tqdm

# fname = "sentences-medium-sample"
fname = "sentences-indeplux-1928-01-01"
with open("indep/" + fname + ".jsonl") as fr:
    lines = fr.readlines()
    
pers_sentences = dict()
for line in tqdm(lines):
    line = literal_eval(line)
    sentence = line["text"]
    sentence_id = line["id"]
    list_entities = []
    detected_entities = pipeline_person(line["text"])
    for entity in reversed(detected_entities):
        if entity["entity_group"] in ["LABEL_1", "LABEL_2"] and entity["score"] > 0.95:
            sentence = sentence[:entity["start"]] + "👀" + sentence[entity["end"] + 1:]
            list_entities.append((entity["start"], entity["end"]))
    list_entities = list(reversed(list_entities))
    sentence = sentence.replace("👀", "[PERS] ")
    if "[PERS]" in sentence:
        pers_sentences[sentence_id] = {"sentence": sentence,
                                       "original_sentence": line["text"],
                                       "entities": list_entities}

In [ ]:
d_processed = []
for pers_sent in pers_sentences:
    sentence = pers_sentences[pers_sent]["sentence"]
    entities = pers_sentences[pers_sent]["entities"]
    processed_sentence = pipeline_hidden_vs_public(sentence)
    labels = get_labels(processed_sentence)
    if len(entities) == len(labels):
        d_processed.append({"id": pers_sent,
                            "text": pers_sentences[pers_sent]["original_sentence"],
                            "pos": entities,
                            "tags": labels})

In [ ]:
import json

with open('indep/output_' + fname + '.jsonl', 'w') as writer:
    for l in d_processed:
        writer.write(json.dumps(l) + "\n")